# Código utilizado no trabalho:

## Importações e variáveis globais:
- dots -> Pontos do banco de dados.
- graph_x_list -> Todos os X possíveis do intercept no gráfico de erro.

In [94]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

#variables
dots = [(.5, 1.4), (2.3, 1.9), (2.9, 3.2)]
# dots = [
#     (0.2, 0.8),
#     (0.4, 1.2),
#     (0.5, 1.4),
#     (0.7, 1.5),
#     (1.0, 1.7),
#     (1.3, 1.8),
#     (1.6, 2.0),
#     (1.8, 2.2),
#     (2.0, 2.3),
#     (2.3, 1.9),
#     (2.5, 2.6),
#     (2.7, 2.9),
#     (2.9, 3.2),
#     (3.2, 3.4)
# ]

graph_x_list = np.linspace(0, 2)

## Funções utilizadas na resolução das questões:

### **get_error_curve**

Essa função calcula todos os valores de **Y (erros)** da curva de erro.  
O cálculo soma os **quadrados dos resíduos** dos pontos para um dado valor de **X (intercept)**. Os resíduos são definidos assim:

- **Predicted Height** = `intercept + (slope * Weight)`  
  (valor (Y do ponto) previsto pela função de gradiente)  
  (sendo o intercept o X do gráfico naquele momento, o slope o peso e o Weight o X do ponto no gráfico de gradiente)

- **Residual** = `(Observed Height – Predicted Height)²`  
  (elevado ao quadrado para evitar anulação entre erros positivos e negativos)  
  (sendo o Observed Height o valor (Y do ponto) original que deve ser encontrado pela função de gradiente)

Assim, a soma dos resíduos gera o erro total naquele ponto, sendo n a quantidade de pontos no gráfico:

$$
E(\text{intercept}) 
= \sum_{i=1}^{n} 
\left( \; \text{ObservedHeight}_i 
- \left( \text{intercept} + \text{slope} \cdot \text{Weight}_i \right) \; \right)^2
$$


Por fim, a função salva todos os erros (Y) do gráfico em uma lista, que será retornada para se desenhar usando a biblioteca MatPlotLib.


In [95]:
def get_error_curve(slope, using_dots):
    y_values = [] #all erros

    for x in graph_x_list:
        y = 0 #current error

        for current_dot in using_dots:
            dot_x = current_dot[0]
            dot_y = current_dot[1]
            y += (dot_y - (x + (slope * dot_x))) ** 2 #summing the residual to the current error
        
        y_values.append(y) #saving the current error to the list
    
    return y_values

### **get_gradient_line**

Essa função calcula todos os valores de **Y (height)** da reta da função de gradiente.  
Como a função de gradiente é uma reta dos valores **Y (height)** previstos, basta calcular o valor previsto de cada **X (weight)** do gráfico:

- **Predicted Height** = `intercept + (slope * Weight)`  
  (valor (Y do ponto) previsto pela função de gradiente)  
  (sendo o intercept o X do gráfico de erro, o slope o peso e o Weight o X do ponto no gráfico)

Por fim, a função salva todos os heights (Y) do gráfico em uma lista, que será retornada para se desenhar usando a biblioteca MatPlotLib.

In [96]:
def get_gradient_line(intercept, slope):
    y_values = []

    for x in np.linspace(0, 4):
        y = intercept + (slope * x) #getting the height (y) using the x (weight) and intercept (error graph x)
        y_values.append(y)
    
    return y_values

### **get_tan**

Essa função calcula todos os valores de **Y (erros)** da **reta tangente** no gráfico de erro, tocando no ponto do **intercept** atual.

Primeiro, ela calcula o **erro atual** \(y_0\) usando os pontos do banco de dados, o intercept atual e o slope da função de gradiente:

$$
E(\text{intercept}) 
= \sum_{i=1}^{n} 
\left( \; \text{ObservedHeight}_i 
- \left( \text{intercept} + \text{slope} \cdot \text{Weight}_i \right) \; \right)^2
$$

Depois disso, para cada \(x_0\) do gráfico (cada intercept possível), ela calcula o valor da **reta tangente** nesse ponto, usando:

- o erro atual \(y_0\)
- o intercept (o ponto onde a tangente encosta no gráfico)
- a derivada do erro em relação ao intercept (a inclinação da tangente)

Assim, para cada \(x_0\), a tangente é calculada por:

$$
\text{tangent}(x_0)
= y_0 + E'(\text{intercept}) \cdot (x_0 - intercept)
$$

Por fim, a função retorna uma **tupla** contendo:

1. o erro atual \(y_0\)
2. a lista com todos os valores de \(y\) da tangente

Esses valores são usados depois pelo MatPlotLib para desenhar a reta tangente no gráfico de erro.


In [97]:
def get_tan(intercept, intercept_derivative, slope, using_dots):
    y_values = []

    y0 = 0 #current error

    for current_dot in using_dots:
        dot_x = current_dot[0]
        dot_y = current_dot[1]
        y0 += (dot_y - (intercept + (slope * dot_x))) ** 2


    for x0 in graph_x_list:
        y = y0 + intercept_derivative * (x0 - intercept) #tangent calculator
        y_values.append(y)
    
    return (y0, y_values)

### **intercept_derivative_calculator**

Essa função calcula a soma das derivadas dos erros de todos os pontos do banco de dados com base no intercept atual e no slope, retornando a derivada que será usada para atualizar o intercept.

A fórmula da derivada usada é:

$$
derivative = \sum_{i=1}^{n} -2 \cdot \left( y_i - (intercept + slope \cdot x_i) \right)
$$


In [98]:
def intercept_derivative_calculator(intercept, slope, using_dots):
    derivative = 0
    for current_dot in using_dots:
        x = current_dot[0]
        y = current_dot[1]
        derivative += -2 * (y - (intercept + (slope * x)))
    
    return derivative

### **slope_derivative_calculator**

Essa função calcula a soma das derivadas dos erros de todos os pontos do banco de dados com base no slope atual, retornando a derivada que será usada para atualizar o slope.

A fórmula da derivada usada é:

$$
derivative = \sum_{i=1}^{n} -2 \cdot x_i \cdot \left( y_i - (intercept + slope \cdot x_i) \right)
$$


In [99]:
def slope_derivative_calculator(intercept, slope, using_dots):
    derivative = 0
    for current_dot in using_dots:
        x = current_dot[0]
        y = current_dot[1]
        derivative += -2 * x * (y - (intercept + (slope * x)))
    
    return derivative

### **gradient_descend**

Essa função executa o **gradiente descendente** para encontrar os melhores valores de **intercept** e **slope** (opcional) que minimizam o erro quadrático total dos pontos. Ela suporta modos:

- **Determinístico**: usa todos os pontos (`using_dots = dots`)  
- **Estocástico**: usa apenas um ponto aleatório por passo  
- **Mini-batch**: usa um lote aleatório de pontos (`batch_size`)  

---

#### Parâmetros

- `learning_rate`: taxa de aprendizado  
- `step_size_intercept`, `step_size_slope`: tamanho do passo atual para intercept e slope  
- `intercept`, `slope`: valores iniciais da reta  
- `derivate_slope`: se True, atualiza também o slope  
- `stochastic`, `mini_batch`: modos de seleção de pontos  
- `batch_size`: tamanho do mini-batch  
- `minimum_step_size`: primeiro critério de parada  
- `max_number_of_steps`: segundo critério de parada (número máximo de passos)  

---

#### Fluxo da função

1. Seleciona os pontos a serem usados (`using_dots`) dependendo do modo.  
2. Calcula a curva de erro atual (`current_curve_y`) com base no slope atual.  
3. Calcula a derivada do **intercept** (`intercept_derivative`).  
4. Obtém o erro atual e a reta tangente para colocar no histórico.  
5. Obtém a linha de gradiente atual para colocar no histórico.
6. Atualiza o **intercept** usando a derivada e a taxa de aprendizado.  
7. Se `derivate_slope` for True, calcula a derivada do slope e atualiza o slope da mesma forma do intercept.  
8. Armazena no histórico (`animation_history`) informações sobre o erro, os passos e os valores atuais e novos de intercept e slope.  
9. Verifica a condição de parada: se os passos forem menores que `minimum_step_size`, interrompe o loop.  

---

#### Retorno

A função retorna uma tupla contendo:

1. Número de iterações realizadas (frames da animação)  
2. `animation_history`: lista com histórico de informações de cada passo  
3. `tangent_list`: lista com histórico da reta tangente de cada passo  
4. `gradient_list`: lista com histórico da reta de gradiente de cada passo 
5. `curve_y_list`: lista com histórico da curva de erro de cada passo

In [100]:
def gradient_descend(   learning_rate = .1, 
                        step_size_intercept = 0, step_size_slope = 0, 
                        intercept = 0, slope = .64, 
                        derivate_slope = False, 
                        stochastic = False, 
                        mini_batch = False, batch_size = 3,
                        minimum_step_size = .001, max_number_of_steps = 1000):
    
    animation_history = []
    tangent_list = []
    gradient_list = []
    curve_y_list = []

    for i in range(max_number_of_steps):
        current_history = []
        
        #-------- USING DOTS --------
        if stochastic: #random dot
            using_dots = [dots[np.random.choice(len(dots))]]

        elif mini_batch: #randoms dots
            batch_indices = np.random.choice(len(dots), size=min(batch_size, len(dots)), replace=False)
            using_dots = [dots[i] for i in batch_indices]

        else: #all dots
            using_dots = dots

        #-------- GETTING THE CURVE --------
        current_curve_y = get_error_curve(slope, using_dots)
        curve_y_list.append(current_curve_y)

        ##-------- INTERCEPT DERIVATIVE --------
        intercept_derivative = intercept_derivative_calculator(intercept, slope, using_dots)

        #-------- GETTING THE TANGENT AND CURRENT ERROR --------
        current_error, current_tangent = get_tan(intercept, intercept_derivative, slope, using_dots)
        current_history.append(current_error) #current error
        tangent_list.append(current_tangent) #current tangent

        #-------- GETTING THE GRADIENT LINE --------
        current_gradient_line = get_gradient_line(intercept, slope)
        gradient_list.append(current_gradient_line) #current gradient

        #-------- UPDATE INTERCEPT --------
        step_size_intercept = intercept_derivative * learning_rate
        current_history.append(step_size_intercept) #intercept step size
        current_history.append(intercept) #old intercept
        intercept -= step_size_intercept
        current_history.append(intercept) #new intercept

        #-------- OPTIONAL: UPDATE SLOPE --------
        if derivate_slope:
            slope_derivative = slope_derivative_calculator(intercept, slope, using_dots)
            step_size_slope = slope_derivative * learning_rate
            current_history.append(step_size_slope) #slope step size
            current_history.append(slope) #old slope
            slope -= step_size_slope
            current_history.append(slope) #new slope
        else:
            step_size_slope = 0
            current_history.append(step_size_slope) #slope step size
            current_history.append(slope) #old slope
            current_history.append(slope) #new slope

        #-------- UPDATE HISTORY --------
        animation_history.append(current_history) #[error, step_size_intercept, current_intercept, new_intercept, step_size_slope, current_slope, new_slope] history

        #-------- STOP CONDITION --------
        if abs(step_size_intercept) < minimum_step_size and (not derivate_slope or abs(step_size_slope) < minimum_step_size):
            break
    
    return (len(animation_history), animation_history, tangent_list, gradient_list, curve_y_list)

### **create_animation**

Essa função cria uma **animação** mostrando o processo do gradiente descendente em duas visualizações simultâneas:

1. **Gráfico do erro**: mostra a curva de erro em função do intercept, a reta tangente no ponto atual e o valor do erro.  
2. **Gráfico do gradiente**: mostra a reta de regressão (linha do slope e intercept) junto com os pontos do banco de dados.

---

#### Parâmetros

- `step_count`: número de iterações da animação  
- `animation_history`: histórico de cada passo contendo erro, passos de intercept e slope, valores antigos e novos  
- `tangent_list`: lista com os valores da reta tangente para cada passo  
- `gradient_list`: lista com os valores da reta de gradiente para cada passo  
- `curve_y_list`: lista da curva de erro para cada passo  

---

#### Configurações dos gráficos

- **Gráfico de erro**:  
  - Linha azul: curva de erro  
  - Ponto azul: erro atual  
  - Linha vermelha: reta tangente no intercept atual  

- **Gráfico do gradiente**:  
  - Pontos azuis: banco de dados (`dots`)  
  - Linha vermelha: reta de gradiente atual  

---

#### Fluxo da animação

1. Para cada frame (passo do gradiente descendente):  
   - Atualiza a **curva de erro**, **ponto da tangente** e **linha tangente** no gráfico de erro  
   - Atualiza o **texto** com os valores do passo de intercept e, se aplicável, do slope  
   - Atualiza os **pontos do banco de dados** e a **linha de regressão** no gráfico de gradiente  

2. Cada frame mostra a situação **antes da atualização do próximo passo**, mostrando a evolução do intercept e do slope (opcional).

---

#### Retorno

A função retorna um objeto `FuncAnimation`, que pode ser exibido diretamente em **Jupyter Notebook** usando `HTML(animation.to_jshtml())`.  

In [101]:
def create_animation(step_count, animation_history, tangent_list, gradient_list, curve_y_list):
    fig, (ax_error, ax_gradient) = plt.subplots(1, 2, figsize=(14, 7))

    #error graph config
    ax_error.set_xlim(0, 2)
    ax_error.set_ylim(0, 4)
    ax_error.set_xticks([i * .25 for i in range(0, 9)])
    ax_error.set_yticks([i * .25 for i in range(0, 17)])
    ax_error.grid(True)
    ax_error.set_xlabel("Intercept", fontsize=12)
    ax_error.set_ylabel("Error", fontsize=12)
    
    #gradient graph config
    ax_gradient.set_xlim(0, 4)
    ax_gradient.set_ylim(0, 4)
    ax_gradient.set_xticks([i * .25 for i in range(0, 17)])
    ax_gradient.set_yticks([i * .25 for i in range(0, 17)])
    ax_gradient.grid(True)
    ax_gradient.set_xlabel("Weight", fontsize=12)
    ax_gradient.set_ylabel("Height", fontsize=12)

    #things on error graph
    error_curve, = ax_error.plot([], [], '-', color='blue')
    tangent_point, = ax_error.plot([], [], 'o', color='blue')
    tangent_line, = ax_error.plot([], [], '-', color='red')

    error_text = ax_error.text(0.010, 0.99, "", transform=ax_error.transAxes, fontsize=12, va='top')

    #things on gradient graph
    gradient_points, = ax_gradient.plot([], [], 'o', color='blue')
    gradient_line, = ax_gradient.plot([], [], '-', color='red')

    #updating the graph
    def update_animation(frame):
        history = animation_history[frame]
        (
            error, 
            step_size_intercept, current_intercept, new_intercept, 
            step_size_slope, current_slope, new_slope) = history

        #error things
        error_curve.set_data(graph_x_list, curve_y_list[frame]) #curve
        tangent_point.set_data([current_intercept], [error]) #tangent point
        tangent_line.set_data(graph_x_list, tangent_list[frame]) #tangent line

        #print text
        print_text = (
            f"Intercept Step Size: {step_size_intercept:.4f}\n"
            f"Old Intercept: {current_intercept:.4f}\n"
            f"New Intercept: {new_intercept:.4f}"
        )

        #changing the slope
        if step_size_slope != 0:
            print_text += (
                f"\n\nSlope Step Size: {step_size_slope:.4f}\n"
                f"Old Slope: {current_slope:.4f}\n"
                f"New Slope: {new_slope:.4f}"
            )
        
        error_text.set_text(print_text)

        #gradient things
        gradient_points.set_data([dot[0] for dot in dots], [dot[1] for dot in dots]) #database points
        gradient_line.set_data(np.linspace(0, 4), gradient_list[frame]) #gradient line

        return tangent_point, tangent_line, error_curve, gradient_points, gradient_line, error_text

    animation = FuncAnimation(fig, update_animation, frames=step_count, interval=100, blit=False)

    plt.close(fig)

    return animation

# **Questão A**

Rodando a função de gradiente com **dois valores diferentes de learning rate** e comparando com o gráfico do slide 4:  

- **Learning rate do gradiente 1**: 0.1  
- **Learning rate do gradiente 2**: 0.2  

**Comparando os resultados:**  

No gráfico do slide 4, é possível observar que os passos do intercept são maiores no início e vão diminuindo ao longo das iterações.  
O mesmo é visto nas duas animações. No entanto, com o learning rate maior no gradiente 2, o passo inicial é muito grande, fazendo com que o intercept passe do mínimo, mas depois ele consegue voltar e localizar o ponto mínimo corretamente.


In [ ]:
gradient_1 = create_animation(*gradient_descend(learning_rate=.1))
gradient_2 = create_animation(*gradient_descend(learning_rate=.2))

anim_list = [gradient_1, gradient_2]
html = "".join(anim.to_jshtml() for anim in anim_list)
HTML(html)

# **Questão B**